# 💾 에이전트 대화 저장 및 재개


에이전트를 서비스 또는 클라이언트 애플리케이션에서 호스팅할 때, 여러 요청이나 세션에 걸쳐 **대화 상태를 유지**해야 하는 경우가 많습니다. Session을 저장하면 대화 컨텍스트를 나중에 다시 불러올 수 있습니다.

---

## 🚀 에이전트 생성 및 스레드 시작

먼저 에이전트를 생성하고 대화 상태를 저장할 새 스레드를 만듭니다.

In [ ]:
from azure.identity import DefaultAzureCredential
from agent_framework import Agent
from agent_framework.azure import AzureOpenAIChatClient

agent = Agent(
    client=AzureOpenAIChatClient(credential=DefaultAzureCredential()),
    name="Assistant",
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨"
)

session = agent.create_session()

---

## 💬 에이전트 실행

에이전트를 실행할 때 스레드를 전달하여 `Session`이 대화 내용을 포함하게 합니다.

In [ ]:
# 에이전트를 실행하고 대화를 세션에 추가합니다.
response = await agent.run("거인에 대한 짧은 농담을 해줘.", session=session)
print(response.text)

---

## 💿 세션을 저장소에 저장하기

### 📝 직렬화 프로세스

1. 세션의 `to_dict()` 메서드를 호출하여 데이터를 딕셔너리로 직렬화
2. JSON으로 변환
3. 데이터베이스, Blob 스토리지 또는 파일에 저장

In [ ]:
import json
import os

# 세션 상태 직렬화
serialized_session = session.to_dict()
serialized_json = json.dumps(serialized_session)

# 예시, 현재 경로에 agent_session.json 파일로 저장하기
# 실제 프로덕션에서는 DB나 Blob 스토리지로 교체하는 것을 권장
current_dir = os.getcwd()
# temp_dir = tempfile.gettempdir()
print(f"현재 작업 디렉토리: {current_dir}")

file_path = os.path.join(current_dir, "agent_session.json")
with open(file_path, "w") as f:
    f.write(serialized_json)

---

## 📂 세션을 저장소에서 불러오기

### 🔄 역직렬화 프로세스

저장소에 저장된 대화를 다시 불러오려면, 저장된 JSON 데이터를 불러와서 `Session` 인스턴스를 다시 생성합니다.

In [ ]:
# 저장된 JSON 불러오기
from agent_framework import AgentSession

with open(file_path, "r") as f:
    loaded_json = f.read()

reloaded_data = json.loads(loaded_json)

# 복원한 세션을 동일한 에이전트 유형에 연결된 Session으로 역직렬화
resumed_session = AgentSession.from_dict(reloaded_data)

---

## ▶️ 대화 재개

불러온 스레드를 사용하여 대화를 자연스럽게 이어갈 수 있습니다.

In [ ]:
# 복원된 세션을 사용하여 대화를 계속합니다.
response = await agent.run("좀 전에 뭐라고 했었지?", session=resumed_session)
print(response.text)

---

## 🎓 실용적인 멀티턴 예제

다음은 여러 상호작용에 걸쳐 컨텍스트를 유지하는 방법을 보여주는 완전한 예제입니다:

### 📋 실습 시나리오

1. **첫 번째 상호작용** - 사용자 이름 저장
2. **두 번째 상호작용** - 에이전트가 이름 기억
3. **직렬화** - 스레드를 저장소에 저장
4. **역직렬화** - 저장된 스레드 불러오기
5. **컨텍스트 확인** - 이전 대화 내용 기억 여부 확인

In [ ]:
from agent_framework import Agent, AgentSession
from agent_framework.azure import AzureAIAgentClient
from azure.identity import AzureCliCredential

agent = Agent(
    client=AzureAIAgentClient(credential=AzureCliCredential()),
    name="Assistant",
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨"
)

# 지속적인 대화를 위해서 세션 생성
session = agent.create_session()

# 첫번째 상호작용
response1 = await agent.run("내 이름은 마이클이야", session=session)
print(f"Agent: {response1.text}")
print("=" * 70)

# 두번째 상호작용 - 에이전트가 이름을 기억함
response2 = await agent.run("내 이름이 뭐였지?", session=session)
print(f"Agent: {response2.text}")  # 마이클이라고 답변해야 함
print("=" * 70)

# 스토리지에 세션 직렬화
serialized = session.to_dict()

# 역직렬화하고 대화를 계속합니다.
resumed_session = AgentSession.from_dict(serialized)
response3 = await agent.run("우리가 무슨 이야기를 했지?", session=resumed_session)

print(f"Agent: {response3.text}")  # 이전 컨텍스트를 기억해야 함
